Problem 01:

In [177]:
import pandas as pd
import csv

In [178]:
def fix_csv_quotes(file_path, output_path):
    with open(file_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', newline='', encoding='utf-8') as outfile:
        reader = infile.readlines()
        writer = csv.writer(outfile)

        for line in reader:
            # Check if the line has mismatched quotes and fix it
            num_quotes = line.count('"')
            if num_quotes % 2 != 0:
                # If odd number of quotes, add a quote at the end
                line = line.strip() + '"\n'
            writer.writerow(next(csv.reader([line])))

In [179]:
# Define the input and output file paths
input_file = '../Data/temp_data.csv'
output_file = '../Data/clean_data.csv'

# Call the function to fix the CSV quotes
fix_csv_quotes(input_file, output_file)


In [180]:
# # Read the fixed CSV file into a list of dictionaries
# data = []
# with open(output_file, 'r', encoding='utf-8') as csvfile:
#     reader = csv.DictReader(csvfile)
#     for row in reader:
#         data.append(row)

# # Create a DataFrame from the list of dictionaries
# df = pd.DataFrame(data)

# print(f"Original number of rows: {len(df)}")
# print(df)

In [181]:
# # Remove the row where the "No" column is 1479
# df = df[df['No'] != '1479']  # Ensure 'No' is treated as string for comparison

# # Reset the index and update the "No" column
# df = df.reset_index(drop=True)
# df['No'] = df.index + 1

# # Print the number of rows after deletion for debugging
# print(f"Number of rows after deletion: {len(df)}")

# # Save the updated DataFrame back to CSV
# df.to_csv(output_file, index=False)

# df = df.drop(columns=[df.columns[-1]])

# print(df) 

In [185]:
# Đọc lại file csv, xoá các dòng bị lỗi 
# (mình nhận thấy số lượng dòng bị lỗi không nhiều ~ 10 dòng nên mình sẽ skip luôn vì tụi mình có nhiều data mà =))))
df = pd.read_csv('../Data/clean_data.csv', on_bad_lines = 'skip')

# Loại bỏ cột No và cột Title không cần thiết 
df = df.drop(columns = df.columns[[0, 2]])

# Load lại những thay đổi vào file clean_data.csv
df.to_csv('../Data/clean_data.csv', index=False)